In [1]:
from lib.network import PoseNet, PoseRefineNet
from lib.loss import Loss
from lib.utils import setup_logger
from datasets.linemod.dataset import PoseDataset as PoseDataset_linemod
import torch
import os
from torch import optim
import numpy as np
import time
from torch.autograd import Variable
%load_ext autoreload
%autoreload 2


In [2]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning) 

In [3]:
num_objects = 5
num_points = 500
outf = 'trained_models/linemod'
log_dir = 'experiments/logs/linemod'
repeat_epoch = 20
decay_margin = 0.016
decay_start = False
lr_rate = 0.3
w_rate = 0.3

In [4]:
estimator = PoseNet(num_points = num_points, num_obj = num_objects)
estimator.cuda();

In [5]:
lr = 0.0001

optimizer = optim.Adam(estimator.parameters(), lr=lr)

In [6]:
dataset_root = "./datasets/linemod/Linemod_preprocessed"
noise_trans = 0.03
refine_start = False
decay_start = False


dataset = PoseDataset_linemod('train', 
                              num_points, 
                              True, 
                              dataset_root, 
                              noise_trans, 
                              refine_start)

workers = 4
dataloader = torch.utils.data.DataLoader(dataset, 
                                         batch_size=1, 
                                         shuffle=True, 
                                         num_workers=workers)

/home/aquancva/DenseFusion/datasets/linemod/dataset.py:66: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.meta[item] = yaml.load(meta_file)


Object 2 buffer loaded
Object 4 buffer loaded
Object 5 buffer loaded
Object 10 buffer loaded
Object 11 buffer loaded


In [7]:
test_dataset = PoseDataset_linemod('test', 
                                   num_points, 
                                   False, 
                                   dataset_root, 
                                   0.0, 
                                   refine_start)
testdataloader = torch.utils.data.DataLoader(test_dataset, 
                                             batch_size=1, 
                                             shuffle=False, 
                                             num_workers=workers)

Object 2 buffer loaded
Object 4 buffer loaded
Object 5 buffer loaded
Object 10 buffer loaded
Object 11 buffer loaded


In [8]:
sym_list = dataset.get_sym_list()
num_points_mesh = dataset.get_num_points_mesh()

In [9]:
print('>>>>>>>>----------Dataset loaded!---------<<<<<<<<\nlength of the training set: {0}\nlength of the testing set: {1}\nnumber of sample points on mesh: {2}\nsymmetry object list: {3}'
      .format(len(dataset), 
              len(test_dataset), 
              num_points_mesh, 
              sym_list))

>>>>>>>>----------Dataset loaded!---------<<<<<<<<
length of the training set: 914
length of the testing set: 515
number of sample points on mesh: 500
symmetry object list: [3, 4]


In [10]:
criterion = Loss(num_points_mesh, sym_list)

In [11]:
start_epoch = 0
nepoch = 4
w = 0.015
batch_size = 8

In [12]:
next(iter(dataloader))[1].shape

torch.Size([1, 1, 500])

In [13]:
best_test = np.Inf
print_every = 50

if start_epoch == 1:
    for log in os.listdir(log_dir):
        if '.ipyn' not in log:
            os.remove(os.path.join(log_dir, log))
st_time = time.time()

for epoch in range(start_epoch, nepoch):
    logger = setup_logger('epoch%d' % epoch, os.path.join(log_dir, 'epoch_%d_log.txt' % epoch))
    mess = 'Train time {0}'.format(time.strftime("%Hh %Mm %Ss", time.gmtime(time.time() - st_time)) + ', ' + 'Training started')
    print(mess)
    logger.info(mess)
    
    train_count = 0
    train_dis_avg = 0.0
    
    estimator.train()
    optimizer.zero_grad()

    for rep in range(repeat_epoch):
        for i, data in enumerate(dataloader, 0):
            points, choose, img, target, model_points, idx = data            
            points, choose, img, target, model_points, idx = Variable(points).cuda(), \
                                                             Variable(choose).cuda(), \
                                                             Variable(img).cuda(), \
                                                             Variable(target).cuda(), \
                                                             Variable(model_points).cuda(), \
                                                             Variable(idx).cuda()
            pred_r, pred_t, pred_c, emb = estimator(img, points, choose, idx)
            loss, dis, new_points, new_target = criterion(pred_r, 
                                                          pred_t, 
                                                          pred_c, 
                                                          target, 
                                                          model_points, 
                                                          idx, 
                                                          points, 
                                                          w, refine_start)
            loss.backward()
            
            train_dis_avg += dis.item()
            train_count += 1
            if train_count % batch_size == 0:
                
                mess = 'Train time {0} Epoch {1} Batch {2} Frame {3} Avg_dis:{4}'.format(
                        time.strftime("%Hh %Mm %Ss", time.gmtime(time.time() - st_time)), 
                        epoch, int(train_count / batch_size), 
                        train_count, 
                        train_dis_avg / batch_size)
                logger.info(mess)
                
                optimizer.step()
                optimizer.zero_grad()
                train_dis_avg = 0
                
                
                if train_count % print_every == 0:
                    print(mess)
            
            if train_count != 0 and train_count % 500 == 0:                    
                torch.save(estimator.state_dict(), '{0}/pose_model_current.pth'.format(outf))
            
    print('>>>>>>>>----------epoch {0} train finish---------<<<<<<<<'.format(epoch))
    
    test_dis = 0.0
    test_count = 0
    estimator.eval()
    
    for j, data in enumerate(testdataloader, 0):
        points, choose, img, target, model_points, idx = data
        points, choose, img, target, model_points, idx = Variable(points).cuda(), \
                                                         Variable(choose).cuda(), \
                                                         Variable(img).cuda(), \
                                                         Variable(target).cuda(), \
                                                         Variable(model_points).cuda(), \
                                                         Variable(idx).cuda()
        pred_r, pred_t, pred_c, emb = estimator(img, points, choose, idx)
        _, dis, new_points, new_target = criterion(pred_r, pred_t, pred_c, 
                                                   target, 
                                                   model_points, idx, points, 
                                                   w, refine_start)
        test_dis += dis.item()
        mess = 'Test time {0} Test Frame No.{1} dis:{2}'.format(time.strftime("%Hh %Mm %Ss", time.gmtime(time.time() - st_time)), test_count, dis)
        logger.info(mess)
        
        test_count += 1
    test_dis = test_dis / test_count
    mess = 'Test time {0} Epoch {1} TEST FINISH Avg dis: {2}'.format(time.strftime("%Hh %Mm %Ss", time.gmtime(time.time() - st_time)), epoch, test_dis)
    print(mess)
    logger.info(mess)
    
    if test_dis <= best_test:
        best_test = test_dis
        torch.save(estimator.state_dict(), '{0}/pose_model_{1}_{2}.pth'.format(outf, epoch, test_dis))
        print(epoch, '>>>>>>>>----------BEST TEST MODEL SAVED---------<<<<<<<<')
        
    if best_test < decay_margin and not decay_start:
        decay_start = True
        lr *= lr_rate
        w *= w_rate
        optimizer = optim.Adam(estimator.parameters(), lr=lr)

Train time 00h 00m 00s, Training started
Train time 00h 00m 10s Epoch 0 Batch 25 Frame 200 Avg_dis:0.09846796281635761
Train time 00h 00m 20s Epoch 0 Batch 50 Frame 400 Avg_dis:0.10915831755846739
Train time 00h 00m 30s Epoch 0 Batch 75 Frame 600 Avg_dis:0.056348265963606536
Train time 00h 00m 40s Epoch 0 Batch 100 Frame 800 Avg_dis:0.0650185199920088
Train time 00h 00m 51s Epoch 0 Batch 125 Frame 1000 Avg_dis:0.10046472167596221
Train time 00h 01m 01s Epoch 0 Batch 150 Frame 1200 Avg_dis:0.07848975597880781
Train time 00h 01m 11s Epoch 0 Batch 175 Frame 1400 Avg_dis:0.06953310989774764
Train time 00h 01m 21s Epoch 0 Batch 200 Frame 1600 Avg_dis:0.04715058021247387
Train time 00h 01m 31s Epoch 0 Batch 225 Frame 1800 Avg_dis:0.04781088943127543
Train time 00h 01m 42s Epoch 0 Batch 250 Frame 2000 Avg_dis:0.07396213919855654
Train time 00h 01m 52s Epoch 0 Batch 275 Frame 2200 Avg_dis:0.03070801473222673
Train time 00h 02m 02s Epoch 0 Batch 300 Frame 2400 Avg_dis:0.04866604146081954
Train 

KeyboardInterrupt: 

In [ ]:
t = next(iter(dataloader))

In [ ]:
points, choose, img, target, model_points, idx = t

In [ ]:
points.shape

In [ ]:
target.shape

In [ ]:
choose.shape

In [ ]:
model_points.shape